In [5]:
import tensorflow as tf
import os 
import sys 
from src.brain_tumor_classification.exception import Custom_Exception
from src.brain_tumor_classification.logger import logging
from tensorflow import keras 

In [6]:
from src.brain_tumor_classification.constants import *
from src.brain_tumor_classification.utils.common import *

In [4]:
os.getcwd()

'/home/googlyji/ml_projects/dl_projects/brain_tumor/brain-tumor-classification'

In [3]:
os.chdir("../")

In [7]:
from dataclasses import dataclass
from pathlib import Path

In [109]:
@dataclass(frozen=True)
class Training_Config:
    root_dir:Path
    trained_model_path:Path
    updated_model_weights:Path
    training_data: Path 
    params_epochs: int 
    params_image_size: list  




@dataclass
class check_:
    root_dir: Path
    tensorboard_dir: Path
    checkpoint: Path

In [110]:
# components 
class Configuration_manager:

    def __init__(self,config_path=CONFIG_PATH,param_path=PARAMS_PATH):
        config_path = Path(config_path)
        param_path = Path(param_path)
        self.config=read_yaml(config_path)
        self.params=read_yaml(param_path)

        create_directories([self.config.artifacts_root])


    def config_check(self)->check_:
        
        config=self.config.prepare_callbacks

        create_directories([config.root_dir])
        create_directories([config.tensorboard_dir])
        create_directories([config.checkpoint])


        data_checkpoint=check_(
            root_dir=config.root_dir,
            tensorboard_dir=config.tensorboard_dir,
            checkpoint=config.checkpoint
            )
        
        return data_checkpoint
    

    def training_config(self)-> Training_Config:
        training=self.config.training
        params=self.params
        prepare_updated_model=self.config.base_model
        training_data=os.path.join("artifacts/data_ingestion/files/bain_tumor/Training")

        create_directories([training.root_dir])


        training_config=Training_Config(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_model_weights=Path(prepare_updated_model.updated_model_weights),
            training_data=Path(training_data), 
            params_epochs=params.epochs, 
            params_image_size=params.Image_size
            )  

        return training_config
    


In [111]:
from datetime import datetime

In [112]:
str(datetime.now().strftime("%s")[8:])

'73'

In [113]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [118]:
# components
class comp_checkpoint_ingestion:
    def __init__(self,config1:check_):
        self.config=config1


    def create_tb_intialization(self):

        a=datetime.now().strftime("%H-%M")

        running_logs=os.path.join(self.config.tensorboard_dir,a)

        return tf.keras.callbacks.TensorBoard(log_dir=running_logs)

    
    def create_chkpt_callbacks(self):

        a=datetime.now().strftime("%s")[8:]

        model_path=os.path.join(self.config.checkpoint,f"model{str(a)}.h5")

        return tf.keras.callbacks.ModelCheckpoint(filepath=model_path,save_best_only=True)

        
    

    def callback_tb_ckt(self):
        
        return [self.create_tb_intialization(),self.create_chkpt_callbacks()]
    




In [119]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow 

In [120]:
class Training:
    def __init__(self,config:Training_Config):
        self.config=config

    
    def get_updated_base_model(self):
        self.model=tf.keras.models.load_model(self.config.updated_model_weights)


    def train_images(self):
        self.train_ds=tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255.,
                validation_split=0.2,
                horizontal_flip=True,rotation_range=0.2,zoom_range=0.2)
        
        self.val_ds=tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.,
            validation_split=0.2
        )
        
        self.train_img=self.train_ds.flow_from_directory(self.config.training_data,target_size=(224,224),
                                                      class_mode="categorical",batch_size=12,seed=1337,
                                                      subset="training")
        
        self.valid_img=self.val_ds.flow_from_directory(self.config.training_data,target_size=(224,224),
                                                 class_mode="categorical",batch_size=12,seed=1337,
                                                 subset="validation")
        

    def train_model(self,callbacks_list:list):
        self.callbacks=callbacks_list
        self.model.fit(self.train_img,
                       epochs=self.config.params_epochs,
                       validation_data=self.valid_img,
                       callbacks=self.callbacks)
        
        self.model.save(self.config.trained_model_path)





In [121]:
a=Configuration_manager()
v=a.config_check()
z=a.training_config()
chk=comp_checkpoint_ingestion(v)
t=chk.callback_tb_ckt()
c=chk.create_chkpt_callbacks()
tc=chk.callback_tb_ckt()
T=Training(z)
T.get_updated_base_model()
T.train_images()
T.train_model(tc)

Found 4571 images belonging to 4 classes.
Found 1141 images belonging to 4 classes.
381/381 [==============================] - 570s 1s/step - loss: 0.6783 - accuracy: 0.7436 - val_loss: 0.7608 - val_accuracy: 0.7082


/home/googlyji/anaconda3/envs/brain/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
import tensorflow as tf
from tensorflow import keras

In [18]:
from tensorflow.keras.models import save_model